信号是一种操作系统的功能，它提供了一种通知事件程序并使其异步处理的方法。
## 接收信号

In [4]:
import signal
import os
import time

def receive_signal(signum, stack):
    print('Received:', signum)
    
signal.signal(signal.SIGUSR1, receive_signal)
signal.signal(signal.SIGUSR2, receive_signal)

print('My pid is ', os.getpid())
while True:
    print('wating...')
    time.sleep(3)

My pid is  3277
wating...
wating...
Received: 31
wating...
Received: 31
wating...


KeyboardInterrupt: 

在另一个窗口中运行
```
$ kill -USR1 $pid
$ kill -USR2 $pid
$ kill -INT $pid
```

## 检索注册的信号处理器

In [5]:
def alarm_received(n, stack):
    return
signal.signal(signal.SIGALRM, alarm_received)
signals_to_names = {getattr(signal, n): n for n in dir(signal) if n.startswith('SIG') and '_' not in n}
for s, name in sorted(signals_to_names.items()):
    handler = signal.getsignal(s)
    if handler is signal.SIG_DFL:
        handler = 'SIG_DFL'
    elif handler is signal.SIG_IGN:
        handler = 'SIG_IGN'
    print(f"{name:<10} ({s:2d}) :", handler)

SIGHUP     ( 1) : SIG_DFL
SIGINT     ( 2) : <built-in function default_int_handler>
SIGQUIT    ( 3) : SIG_DFL
SIGILL     ( 4) : SIG_DFL
SIGTRAP    ( 5) : SIG_DFL
SIGIOT     ( 6) : SIG_DFL
SIGEMT     ( 7) : SIG_DFL
SIGFPE     ( 8) : SIG_DFL
SIGKILL    ( 9) : None
SIGBUS     (10) : SIG_DFL
SIGSEGV    (11) : SIG_DFL
SIGSYS     (12) : SIG_DFL
SIGPIPE    (13) : SIG_IGN
SIGALRM    (14) : <function alarm_received at 0x10a4f3b70>
SIGTERM    (15) : SIG_DFL
SIGURG     (16) : SIG_DFL
SIGSTOP    (17) : None
SIGTSTP    (18) : SIG_DFL
SIGCONT    (19) : SIG_DFL
SIGCHLD    (20) : SIG_DFL
SIGTTIN    (21) : SIG_DFL
SIGTTOU    (22) : SIG_DFL
SIGIO      (23) : SIG_DFL
SIGXCPU    (24) : SIG_DFL
SIGXFSZ    (25) : SIG_IGN
SIGVTALRM  (26) : SIG_DFL
SIGPROF    (27) : SIG_DFL
SIGWINCH   (28) : SIG_DFL
SIGINFO    (29) : SIG_DFL
SIGUSR1    (30) : <function receive_signal at 0x10a4f3c80>
SIGUSR2    (31) : <function receive_signal at 0x10a4f3c80>


使用 getsignal() 函数可以查看某个信号注册了哪个信号处理器。传入信号编号作为参数，返回值是注册的信号处理器，或者特殊值 SIG_IGN （如果信号被忽略），SIG_DFL（默认信号处理行为），或者 None （如果存在的信号处理器是从 C 注册的）。

发送信号的函数是os.kill()。

## 警报
警报是一种特殊的信号，程序要求操作系统在一段时间之后再去通知它。

In [21]:
import time
def receive_alarm(signum, stack):
    print(signum)
    time.sleep(1)
    print('Alarm :', time.ctime())

signal.signal(signal.SIGALRM, receive_alarm)
signal.alarm(2)
print('Before:', time.ctime())
time.sleep(6)  # 调用过程中信号进入执行
print('After:', time.ctime())

Before: Wed Sep 26 11:07:15 2018
14
Alarm : Wed Sep 26 11:07:18 2018
After: Wed Sep 26 11:07:21 2018


## 忽略信号

In [11]:
def do_exit(sig, stack):
    raise SystemExit('Exiting')
    
signal.signal(signal.SIGINT, signal.SIG_IGN)
signal.signal(signal.SIGUSR1, do_exit)

print('My PID:', os.getpid())
signal.pause()

My PID: 3277


SystemExit: Exiting

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


正常的 SIGINT （通过 Ctrl-C 由shell 发送给 程序的信号）会引发 KeyboardInterrupt。 这个例子中忽略了 SIGINT 然而当接收到信号 SIGUSR1 时会引起 SystemExit

## 信号和线程
信号和线程通常不会很好结合在一起，因为只有进程的主线程才会接受信号。下列的例子设置了一个信号处理方法，在一个线程中等待信号到达，然后从另一个线程中发送信号。

In [22]:
import threading

def signal_handler(num, stack):
    print(f'Receive signal {num} in {threading.currentThread().name}')
    
signal.signal(signal.SIGUSR1, signal_handler)

def wait_for_signal():
    print('Wait for signal in ', threading.currentThread().name)
    signal.pause()
    print('Done waiting')
    
# 启动一个不会接收信号的线程
receiver = threading.Thread(target=wait_for_signal, name='receiver')
receiver.start()
time.sleep(0.1)

def send_signal():
    print('Sending signal in ', threading.currentThread().name)
    os.kill(os.getpid(), signal.SIGUSR1)
    
sender = threading.Thread(target=send_signal, name='sender')
sender.start()
sender.join()

# 等待线程看到信号（不会发生）
print('Waiting for ', receiver.name)
signal.alarm(2)
receiver.join()

Wait for signal in  receiver
Sending signal in  sender
Receive signal 30 in MainThread
Waiting for  receiver
14
Alarm : Wed Sep 26 11:13:53 2018


KeyboardInterrupt: 

信号处理程序都在主线程中注册，因为这是 Python 的 signal 模块实现的要求，无论底层平台是否支持线程和信号混合开发。尽管接收线程调用了 signal.pause()，但是它不会接收到信号。脚本结束位置的 signal.alarm(2) 阻止了无限循环，否则接收者线程永远不会退出。

In [23]:
def signal_handler(num, stack):
    print(time.ctime(), 'Alarm in ',threading.currentThread().name)

signal.signal(signal.SIGALRM, signal_handler)

def use_alarm():
    t_name = threading.currentThread().name
    print(time.ctime(), 'Seting alarm in ', t_name)
    signal.alarm(1)
    print(time.ctime(), 'Sleep in ', t_name)
    time.sleep(3)
    print(time.ctime(), 'Done with sleep ', t_name)
    
alarm_thread = threading.Thread(target=use_alarm, name='alarm_thread')
alarm_thread.start()
time.sleep(0.1)

print(time.ctime(), 'Wait for  ',alarm_thread.name)
alarm_thread.join()
print(time.ctime(), 'Exiting normally  ')

Wed Sep 26 13:47:20 2018 Seting alarm in  alarm_thread
Wed Sep 26 13:47:20 2018 Sleep in  alarm_thread
Wed Sep 26 13:47:20 2018 Wait for   alarm_thread
Wed Sep 26 13:47:21 2018 Alarm in  MainThread
Wed Sep 26 13:47:23 2018 Done with sleep  alarm_thread
Wed Sep 26 13:47:23 2018 Exiting normally  
